In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
from tqdm import tqdm, trange, tqdm_notebook
import re
import nltk
from multiprocessing import cpu_count, Pool
from functools import partial
import ipywidgets
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Use to delete auxiliary columns that should not be passed to a model
def del_str_cols(df): # df should be X, e.g. X_train or X_dev
    del df["articleBody"]
    del df["Headline"]
    del df["Body ID"]
    for col_name in df.columns:
        if "___" == col_name[0:3]:
            del df[col_name]

In [2]:
X_train = pd.read_hdf("X_train_disjoint-allfeatures.h5", key="df")
y_train = pd.read_hdf("y_train_disjoint.h5", key="df")

X_dev = pd.read_hdf("X_dev_disjoint-allfeatures.h5", key="df")
y_dev = pd.read_hdf("y_dev_disjoint.h5", key="df")

#X_test = pd.read_hdf("X_TEST-allfeatures.h5", key="df")
#y_test = pd.read_hdf("y_TEST_disjoint.h5", key="df")

In [3]:
X_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,0
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,1,0,0,0,0,0,0,0,0,0
5,'Nasa Confirms Earth Will Experience 6 Days of...,Thousands of people have been duped by a fake ...,nasa confirms earth will experience 6 days of ...,"[nasa, confirms, earth, will, experience, 6, d...",thousands of people have been duped by a fake ...,"[thousand, of, people, have, been, duped, by, ...",17,15,14,0.074236,...,3,0,0,0,0,0,0,0,0,0
6,Accused Boston Marathon Bomber Severely Injure...,A British fighter who travelled to Iraq to sto...,accused boston marathon bomber severely injure...,"[accused, boston, marathon, bomber, severely, ...",a british fighter who travelled to iraq to sto...,"[a, british, fighter, who, travelled, to, iraq...",4,1,1,0.004608,...,2,0,0,0,0,0,0,0,0,0
7,Identity of ISIS terrorist known as 'Jihadi Jo...,"Adding to Apple's iOS 8 launch troubles, a rep...",identity of isis terrorist known as jihadi joh...,"[identity, of, isi, terrorist, known, a, jihad...",adding to apple s ios 8 launch troubles a repo...,"[adding, to, apple, s, io, 8, launch, trouble,...",2,1,0,0.011111,...,2,0,0,0,0,0,0,0,0,0


In [4]:
y_train.head()

0    unrelated
2    unrelated
5        agree
6    unrelated
7    unrelated
Name: Stance, dtype: object

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

(44974, 43)
(44974,)
(4998, 43)
(4998,)


In [10]:
X_train.head()

,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,wrf_hl_fake,wrf_hl_fraud,wrf_hl_hoax,wrf_hl_false,wrf_hl_deny,wrf_hl_denies,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
44610,9,8,4,0.057377,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
10499,4,3,1,0.023256,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
43957,8,5,4,0.024465,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
49559,4,2,1,0.009926,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1851,1,1,0,0.008403,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [12]:
clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)

In [13]:
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1       44003.6782           44.82s
         2       39386.2805           42.36s
         3       35647.5327           41.60s
         4       32591.8800           43.49s
         5       30074.0478           43.13s
         6       27969.2272           41.99s
         7       26212.1162           41.59s
         8       24722.2751           42.17s
         9       23476.4802           41.41s
        10       22386.3081           41.45s
        20       17304.8224           37.61s
        30       15906.0471           36.34s
        40       15296.5875           33.05s
        50       14989.3720           29.87s
        60       14763.3830           27.56s
        70       14602.0671           25.17s
        80       14457.9762           22.97s
        90       14333.8437           20.82s
       100       14229.1701           19.00s
       200       13507.8045            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=14128, subsample=1.0,
              verbose=True, warm_start=False)

In [14]:
preds = clf.predict(X_dev)

In [16]:
actual = y_dev

In [17]:
from sklearn.metrics import classification_report

In [19]:
# I checked this against the baseline implementation -- numbers look correct!
print(classification_report(actual, preds))

             precision    recall  f1-score   support

      agree       0.55      0.16      0.24       331
   disagree       0.40      0.02      0.04       102
    discuss       0.65      0.84      0.73       876
  unrelated       0.96      0.98      0.97      3689

avg / total       0.87      0.88      0.86      4998



In [30]:
#crf = sklearn.ensemble.RandomForestClassifier(n_estimators=200, n_jobs=8, random_state=42)

In [31]:
#crf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=8,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [39]:
preds = clf.predict(X_dev)
print(classification_report(y_dev, preds))

             precision    recall  f1-score   support

      agree       0.55      0.16      0.24       331
   disagree       0.40      0.02      0.04       102
    discuss       0.65      0.84      0.73       876
  unrelated       0.96      0.98      0.97      3689

avg / total       0.87      0.88      0.86      4998



In [40]:
y_train.value_counts(normalize=True)

unrelated    0.730555
discuss      0.178614
agree        0.074421
disagree     0.016409
Name: Stance, dtype: float64

In [41]:
# 4-class problem:
# By guessing most common category (unrelated), we would achieve approx. 73% accuracy. The baseline model achieves 88%.

In [62]:
def score_4class(actual, predicted):
    assert actual.shape[0] == predicted.shape[0]
    def calc_score_helper(actual, predicted):
        score = 0.0
        for actual_s, pred_s in zip(actual, predicted):
            if actual_s == pred_s:
                score += 0.25
                if actual_s != "unrelated":
                    score += 0.50
            if (actual_s != "unrelated") and (pred_s != "unrelated"):
                score += 0.25
        return score
    score = calc_score_helper(actual, predicted)
    max_score = calc_score_helper(actual, actual)
    return score, max_score

In [63]:
score, max_score = score_4class(y_dev, preds)
weighted_acc = score/max_score
print("Weighted acc.: "+str(weighted_acc)+" ("+str(score)+" out of "+str(max_score)+")")

Weighted acc.: 0.7975350140056022 (1779.5 out of 2231.25)


In [ ]:
print(preds)